In [ ]:
%%capture

!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu121
!pip install transformers datasets accelerate peft trl bitsandbytes
!pip install unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer, SFTConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# Load the model and tokenizer with 4-bit quantization for memory efficiency
max_seq_length = 2048
dtype = None # Auto-detection
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    load_in_8bit = False
)

In [ ]:
# Apply Low-Rank Adaptation (LoRA) to reduce the number of trainable parameters
model = FastLanguageModel.get_peft_model(
    model,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    r = 8,
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="pharmacy_raft_dataset.jsonl", split="train")

In [ ]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [ ]:
# Format the RAFT dataset using a chat template for the trainer
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    contexts     = examples["context"]
    outputs      = examples["answer"]
    texts = []
    for instruction, context, output in zip(instructions, contexts, outputs):
        messages = [
            {"role": "user", "content": f"Context: {context}\n\nQuestion: {instruction}"},
            {"role": "model", "content": output},
        ]
        text = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
        texts.append(text)
    return { "text" : texts }

dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
# Configure the SFTTrainer with optimized hyperparameters for the 8B model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    max_seq_length = max_seq_length,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.59 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

383.5385 seconds used for training.
6.39 minutes used for training.
Peak reserved memory = 6.883 GB.
Peak reserved memory for training = 1.293 GB.
Peak reserved memory % of max memory = 46.693 %.
Peak reserved memory for training % of max memory = 8.771 %.


In [ ]:
#model.save_pretrained_gguf("model_gguf", tokenizer, quantization_method = "q4_k_m")

In [ ]:
FastLanguageModel.for_inference(model)

# Define a test question
instruction = "Explain the mechanism of action of Warfarin and its main interactions."
context = "Warfarin inhibits vitamin K epoxide reductase. It interacts with cytochrome P450."

In [ ]:
messages = [
    {"role": "user", "content": f"Context: {context}\n\nQuestion: {instruction}"},
]

In [ ]:
prompt = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False,
)

print("Generated prompt:")
print(prompt)
print(type(prompt))

Prompt generado:
<bos><start_of_turn>user
Contexto: La warfarina inhibe la epóxido reductasa de la vitamina K. Interacciona con el citocromo P450.

Pregunta: Explica el mecanismo de acción de la Warfarina y sus principales interacciones.<end_of_turn>
<start_of_turn>model

<class 'str'>


In [ ]:
inputs = tokenizer(
    text=prompt,
    return_tensors="pt",
    padding=True,
).to("cuda")

print(f"Shape: {inputs['input_ids'].shape}")
print(f"Device: {inputs['input_ids'].device}")

Shape: torch.Size([1, 64])
Device: cuda:0


In [ ]:
# 4. Generation
outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
response = tokenizer.tokenizer.batch_decode(outputs)

print("--- ASSISTANT´S ANSWER ---")
print(response[0].split("<start_of_turn>model\n")[-1])

--- RESPUESTA DEL ASISTENTE ---
Para explicar el mecanismo de acción de la warfarina y sus interacciones, es crucial comprender su forma de inhibir la vitamina K y su interacción con el citocromo P450, y de esta forma, cómo afecta la producción de factores de coagulación de la sangre.

Mecanismo de Acción:

La warfarina es un fármaco antiagregante plaquetario que actúa principalmente inhibiendo la vitamina K epóxido reductasa, una enzima crucial en el metabolismo de la vitamina K. Los factores de coagulación de la sangre, como la II, VII, IX y X, se derivan de la vitamina K. La vitamina K epóxido reductasa es esencial para activar estos factores, y es necesario para su carboxilación, una reacción que los hace funcionales como cofactores de coagulación. Al inhibir esta enzima, la warfarina bloquea la carboxilación de estos factores, lo que conduce a la producción de factores de coagulación inactivos. Esto inhibe la cascada de coagulación, previniendo la formación de coágulos sanguíneos.

In [ ]:
# Save the Fine-Tuned LoRA adapters locally
nombre_carpeta = "lora_pharma"
model.save_pretrained(nombre_carpeta)
tokenizer.save_pretrained(nombre_carpeta)

# 2.Zip the folder
import shutil
shutil.make_archive(nombre_carpeta, 'zip', nombre_carpeta)

# Download the zipped folder
from google.colab import files
files.download(f"{nombre_carpeta}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>